<a href="https://colab.research.google.com/github/flora0110/podcast_segment_headine/blob/main/similarity_ok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 42.7 MB/s 
     |████████████████████████████████| 1.2 MB 60.6 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 6.6 MB 55.9 MB/s 
     |████████████████████████████████| 895 kB 56.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=369d7eaffe17cd917457d7246ee3a251598d7cf99ca2be25fb284a06bf0b56b2
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from sentence_transformers import SentenceTransformer, util
#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [30]:
def similarity(podcastDict, sampleDict):
  print(len(sampleDict))
  arr = []
  #index = 0
  podcastN = len(podcastDict)-1
  sampleN = len(sampleDict)-1
  print(sampleN)
  for i in range(podcastN):
    #print(i)
    start = podcastDict[i]['time']
    end = podcastDict[i+1]['time']
    #print(arr)
    #print("start: "+str(start)+" end: "+str(end))
    for j in range(sampleN):
      #print("j: "+str(j))
      if(start>=sampleDict[j]['time'] and end<=sampleDict[j+1]['time']):
        #print("continue "+str(j))
        #print(sampleDict[j]['time'])
        #print(sampleDict[j+1]['time'])
        arr.append({'start_match':j,'ratio_start':(end-start)/(sampleDict[j+1]['time']-sampleDict[j]['time']),'end_match':-1,'ratio_end':-1})
        #print(len(arr))
        #index+=1
        break
      elif(start>=sampleDict[j]['time'] and start<=sampleDict[j+1]['time']):
        #print("not con "+str(j))
        #print(sampleDict[j]['time'])
        #print(sampleDict[j+1]['time'])
        arr.append({'start_match':j,'ratio_start':(sampleDict[j+1]['time']-start)/(sampleDict[j+1]['time']-sampleDict[j]['time'])})
        #print(len(arr))
        for k in range(j,sampleN):
          #print("kkkk "+str(k))
          #print(sampleDict[k]['time'])
          #print(sampleDict[k+1]['time'])
          if(end<=sampleDict[k+1]['time'] and end>=sampleDict[k]['time']):
            #print("update")
            arr[len(arr)-1].update({'end_match':k,'ratio_end':(end-sampleDict[k]['time'])/(sampleDict[k+1]['time']-sampleDict[k]['time'])})
            #print(len(arr))
            #index+=1
        break

  #print(arr)
  #for a in arr:
  #  print(a)
  print(len(podcastDict))
  print(len(sampleDict))
  print(len(arr))
  scores = []
  for i in range(0,len(arr)):
    #print(i)
    if(arr[i]['end_match']==-1):
      embedding_1= model.encode(podcastDict[i]['headline'], convert_to_tensor=True)
      embedding_2 = model.encode(sampleDict[arr[i]['start_match']]['headline'], convert_to_tensor=True)
      #print(podcastDict[i]['headline'])
      #print(sampleDict[arr[i]['start_match']]['headline'])
      score_temp = float(util.pytorch_cos_sim(embedding_1, embedding_2))
      score_temp*=arr[i]['ratio_start']
      scores.append(score_temp)
      #print(score_temp)
    else:

      embedding_1= model.encode(podcastDict[i]['headline'], convert_to_tensor=True)
      embedding_2 = model.encode(sampleDict[arr[i]['start_match']]['headline'], convert_to_tensor=True)
      score_temp = float(util.pytorch_cos_sim(embedding_1, embedding_2))
      score_temp*=arr[i]['ratio_start']
      score_temp_sum=score_temp
      for j in range(arr[i]['start_match']+1,arr[i]['end_match']):
        embedding_1= model.encode(podcastDict[i]['headline'], convert_to_tensor=True)
        embedding_2 = model.encode(sampleDict[j]['headline'], convert_to_tensor=True)
        score_temp = float(util.pytorch_cos_sim(embedding_1, embedding_2))
        score_temp_sum+=score_temp
      embedding_1= model.encode(podcastDict[i]['headline'], convert_to_tensor=True)
      embedding_2 = model.encode(sampleDict[j]['headline'], convert_to_tensor=True)
      score_temp = float(util.pytorch_cos_sim(embedding_1, embedding_2))
      score_temp*=arr[i]['ratio_end']
      score_temp_sum+=score_temp
      scores.append(score_temp_sum/(arr[i]['end_match']-arr[i]['start_match']+1))
  print(scores)
  sum = 0
  for score in scores:
    sum +=score
  print(sum/len(sampleDict))

# social

In [38]:
sampleDict = [{'time': 0.0, 'headline': 'Social Bonding: Child-Parent, Romantic, Friendship, Breakups'}, {'time': 175.0, 'headline': 'ROKA, Athletic Greens, Headspace'}, {'time': 428.0, 'headline': 'Social Bonding as a Biological Process'}, {'time': 603.0, 'headline': 'Social Isolation'}, {'time': 812.0, 'headline': 'Social Homeostasis & Neural Circuits for Social Drive '}, {'time': 1135.0, 'headline': 'Brain Areas & (Neuro)Chemistry of Social Drive '}, {'time': 1368.0, 'headline': 'What is Social Homeostasis & Dopamine'}, {'time': 1620.0, 'headline': 'When We Lack Social Interactions: Short- Versus Long-Term'}, {'time': 1690.0, 'headline': 'Introverts & Extroverts'}, {'time': 1860.0, 'headline': '"Good" Versus "Bad" Social Interactions & Hierarchies'}, {'time': 2034.0, 'headline': 'Loneliness & Dorsal Raphe Nucleus & Social Hunger '}, {'time': 2253.0, 'headline': 'Tools'}, {'time': 2285.0, 'headline': 'Socializing & Food Appetite: Crossover Craving'}, {'time': 2565.0, 'headline': 'Falling in Love'}, {'time': 2705.0, 'headline': 'Tools for Social Bonds: Merging Physiologies; Story'}, {'time': 3234.0, 'headline': 'Childhood Attachment Patterns in Adulthood'}, {'time': 3825.0, 'headline': 'Attachment Styles: Autonomic Versus Intellectual Attachment'}, {'time': 3970.0, 'headline': 'Emotional Empathy & Cognitive Empathy, Arguing'}, {'time': 4185.0, 'headline': 'Allan N. Schore & "Right Brain Psychotherapy"'}, {'time': 4240.0, 'headline': 'Oxytocin & Trust, In Males Versus Females, Hormonal Glue'}, {'time': 4570.0, 'headline': 'Repairing Broken Bonds to Self & Others'}, {'time': 4736.0, 'headline': 'Social (Media) Butterflies: Biological Basis'}, {'time': 5048.0, 'headline': 'Key Points for Bonding & Understanding Social Bonds'}, {'time': 5227.0, 'headline': 'Breaking Up'}, {'time': 5316.0, 'headline': 'Synthesis '}, {'time': 5477.0, 'headline': 'Zero-Cost Support, Sponsors, Patreon, Thorne, Instagram, Twitter'},{'time':5596.9,'headline':""}]


#0.5 not dele

In [50]:
#podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]
#podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]
#podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]
podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]

In [51]:
similarity(sampleDict,podcastDict)

46
45
27
46
25
[0.018658763423540958, 0.08208069683681998, 0.05339618959143983, 0.04056734047238806, 0.22446321726299787, 0.1864418546316589, 0.031053371264989846, 0.2577594544930074, 0.06420580514747933, 0.032529187047373634, 0.04070488827620514, -0.00026563274637221514, 0.09112313007580154, 0.134130311162976, 0.1611819586179194, 0.20082982758680878, 0.12691268144823983, 0.2306863266301242, -7.646688606712169e-05, 0.15384305223189215, 0.024315105637765382, 0.19057990236749858, 0.21076274676169304, 0.03396446962263871, 0.09295047452484308]
0.05832170990181875


## 0.5 dele spon

In [52]:
podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace - The Best Deals'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]

In [53]:
similarity(sampleDict,podcastDict)

46
45
27
46
25
[0.018658763423540958, 0.08122123012422545, 0.05339618959143983, 0.04056734047238806, 0.22446321726299787, 0.1864418546316589, 0.031053371264989846, 0.2577594544930074, 0.06420580514747933, 0.032529187047373634, 0.04070488827620514, -0.00026563274637221514, 0.09112313007580154, 0.134130311162976, 0.1611819586179194, 0.20082982758680878, 0.12691268144823983, 0.2306863266301242, -7.646688606712169e-05, 0.15384305223189215, 0.024315105637765382, 0.19057990236749858, 0.21076274676169304, 0.03396446962263871, 0.09295047452484308]
0.05830302584284929


## 0.6 not dele

In [48]:
podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' The Brain and Nervous System and How to Boost Your Social Bond'}, {'time': 121.8, 'headline': ' Roca Eyeglasses and Sunglasses - The Best Eyeglasse'}, {'time': 237.8, 'headline': ' Headspace Special Offer - Get Your Athletic Greens Today'}, {'time': 429.0, 'headline': ' The Biology of Social Bonding'}, {'time': 610.7, 'headline': ' The Biology of Social Isolation'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1188.1, 'headline': ' The Sensory System of Stasis'}, {'time': 1209.1, 'headline': ' Dopamine Neurons in the Dorsal Hypothalamus'}, {'time': 1454.0, 'headline': ' Dopamine and the Dopamine Craving'}, {'time': 1660.5, 'headline': ' Fasting - Is it a Good Idea?'}, {'time': 1677.7, 'headline': ' The Social Homeostasis Circuit'}, {'time': 1868.3, 'headline': ' The Hypothalamus and the Brain Centers of the Brain'}, {'time': 1895.5, 'headline': ' The Prefrontal Cortex Is Responsible For Creating a Social Homeostas'}, {'time': 1977.3, 'headline': ' The Dopamine Neurons of the Dorsal Raffi Nucleus'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' How to Develop a Social Bond'}, {'time': 3156.1, 'headline': ' The Final Common Pathway'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Soco , Unconscious or Unconscious'}, {'time': 3434.5, 'headline': " When You're Born?"}, {'time': 3451.2, 'headline': ' The Right Brain System Is a Parent - Child Interaction'}, {'time': 3593.5, 'headline': ' The Left Brain System Is Engaged in Parenting'}, {'time': 3727.6, 'headline': ' Children and the Cognitive Development of Children'}, {'time': 3745.0, 'headline': " Bonding in the Child's Brain"}, {'time': 3879.6, 'headline': ' Cognitive Connections'}, {'time': 3917.5, 'headline': ' Empathy is the ability to Feel or at least Sink'}, {'time': 4014.4, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' Alan Shore on the Podcast?'}, {'time': 4240.8, 'headline': ' Activation of a Set of Neurons'}, {'time': 4262.3, 'headline': ' Oxytocin and Its Effects on the Body'}, {'time': 4392.7, 'headline': ' Lactation and Milk Let - Down Reflex Lactation'}, {'time': 4409.9, 'headline': ' Oxytocin Released in Females and Males'}, {'time': 4492.4, 'headline': ' The Effect of Oxytocin Release on the Body of the Other'}, {'time': 4534.0, 'headline': " MDMA and the 'Bad' Bonding Effects"}, {'time': 4617.7, 'headline': ' The Relationship Between Oxytocin and MDMA'}, {'time': 4915.7, 'headline': ' The Social Interactions of the Child Parent'}, {'time': 5031.7, 'headline': ' Social Bonding - The Basics'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast'}, {'time': 5512.2, 'headline': ' Thorne Supplements - What Are They?'}, {'time': 5574.7, 'headline': " Psychic Neuroscience - What's Next?"},{'time':5596.9,'headline':""}]

In [49]:
similarity(sampleDict,podcastDict)

46
45
27
46
25
[0.018658763423540958, 0.08208069683681998, 0.05339618959143983, 0.04056734047238806, 0.22446321726299787, 0.1864418546316589, 0.031053371264989846, 0.2577594544930074, 0.06420580514747933, 0.032529187047373634, 0.04070488827620514, -0.00026563274637221514, 0.09112313007580154, 0.134130311162976, 0.1611819586179194, 0.20082982758680878, 0.12691268144823983, 0.2306863266301242, -7.646688606712169e-05, 0.15384305223189215, 0.024315105637765382, 0.19057990236749858, 0.21076274676169304, 0.03396446962263871, 0.09295047452484308]
0.05832170990181875


## 0.6 dele spon

In [45]:
podcastDict = [{'time': 0.2, 'headline': ' The Biology Psychology Practices of Social Bonding'}, {'time': 56.9, 'headline': ' Breaking Up With a Romantic Partner'}, {'time': 164.4, 'headline': ' The Biology of Social Bonding'}, {'time': 825.7, 'headline': ' The Homeostasis of the Homeostatic Circuit'}, {'time': 933.8, 'headline': ' The Social Homeostasis Control Center and Effector'}, {'time': 1164.5, 'headline': ' The Social Homeostasis Circuit - The Dorsal Amygd'}, {'time': 1360.4, 'headline': ' The Social Homeostasis Circuit in the Human Brain'}, {'time': 2308.8, 'headline': ' The Homeostatic Drive of the Human Brain'}, {'time': 2380.8, 'headline': ' Dopamine and the Relationships That We Have With Food'}, {'time': 2667.0, 'headline': ' The Relationship Between the Body and the Mind'}, {'time': 3168.4, 'headline': ' The Relationship , the Relationship'}, {'time': 3234.3, 'headline': ' The Human Lab Podcast '}, {'time': 3367.8, 'headline': ' The Autonomic Nervous System of the Infant Mother'}, {'time': 3580.3, 'headline': ' The Left Brain System and Parent - Child Relationships'}, {'time': 3873.9, 'headline': ' Cognitive Connections - How to Have a Deep Emotional Connection'}, {'time': 3917.5, 'headline': ' The Relationship Between Child and Caregivers'}, {'time': 4217.3, 'headline': ' The Effects of Oxytocin on the Body and the Brain'}, {'time': 4908.2, 'headline': ' The Neuroendocrinology of Social Bonding'}, {'time': 5124.7, 'headline': ' The Importance of Social Bonding'}, {'time': 5475.4, 'headline': ' The Human Lab Podcast - Andrew Huberman'},{'time':5596.9,'headline':""}]


In [47]:
similarity(sampleDict,podcastDict)

21
20
27
21
25
[0.24390919124162558, 0.02316946075612652, 0.27781183846181084, 0.026677390979703137, 0.26904348559268937, 0.16060485210362388, 0.05565689478379067, 0.04967944134845355, 0.021490701130551657, 0.06237276878878819, 0.016232776722290603, 7.156862860778432e-05, 0.0909805494342847, 0.10903693668306252, 0.11940868630355125, 0.15557314952051593, 0.07090860824930098, 0.022837106511149292, 0.057591799091785544, 0.10549453150641132, 0.0669632929963476, 0.11699242501478344, 0.21076274676169304, 0.03245198492093778, 0.14028362404907638]
0.11933361007528388
